In [ ]:
import pandas as pd
import json
from pprint import pprint


# 2 input files are needed
family_rules_filepath = "Current_TF_family_rules.xlsx"
annotations_filepath = "all_maize_pfam.json"

# download links for the input files above
#https://github.com/grotewold-lab/grassius-db-builder/blob/main/inputs/private_inputs/Current_TF_family_rules.xlsx?raw=true
#https://github.com/grotewold-lab/grassius-db-builder/blob/main/inputs/private_inputs/all_maize_pfam.json.tar.gz?raw=true


output_filepath = "applied_rules.csv"


In [ ]:
# load family rules
df = pd.read_excel(family_rules_filepath)


# convert the content of one required/forbidden cell into a list of accession names 
# e.g. "PF00249#2" -> ["PF00249","PF00249"]
# e.e. "PF00319#1:PF01486#1" -> ["PF00319","PF01486"]
def parse_rule( cell_content ):
    if not isinstance(cell_content,str):
        return []
    result = []
    for part in cell_content.split(":"):
        if len(part.strip()) == 0:
            continue
        prefix,suffix = part.split("#")
        if suffix == "1":
            result.append(prefix)
        elif suffix == "2":
            result += [prefix,prefix]
    return result


# convert all required and forbidden cells
for row in df.index:
    for col in ['Required','Forbidden']:
        df.at[row,col] = parse_rule( df.loc[row,col] )
        
# get a master list of all domains involved in the 'Required' column
all_required_domains = []
for row in df.index:
    for entry in df.loc[row,'Required']:
        all_required_domains.append(entry)
all_required_domains = set(all_required_domains)

        
# give the dataframe a more descriptive variable name
rules_df = df

In [ ]:
# EXAMPLE
# check the required and forbidden domains for one family

family_name = "MYB"


matching_row = rules_df.loc[rules_df['GRASSIUS'] == family_name].index[0]
for col in ['Required','Forbidden']:
    domains = rules_df.loc[matching_row,col]
    print( f"{col}: {domains}")

In [ ]:
# load domain annotations
with open(annotations_filepath, 'r') as f:
    raw_anno = json.load( f )


# convert domain annotations into lists of accession names
# ignore transcripts without annotations
anno = {}
for tid,tanno in raw_anno.items():
    anno[tid] = []
    if not isinstance(tanno,list):
        tanno = [tanno]
    for entry in tanno:
        acc = entry['@acc'].split(".")[0]
        if isinstance(entry['domains'],list):
            for d in entry['domains']:
                anno[tid].append( acc )
        else:
            anno[tid].append( acc )
            

In [ ]:
raw_anno['Zm00001eb058920_P001']

In [ ]:
# EXAMPLE
# check the raw annotations for a transcript

tid = 'Zm00001eb058920_P001'
tid = tid.replace('_T','_P')

pprint(raw_anno[tid])

In [ ]:
# EXAMPLE
# check the domains present in a transcript

tid = 'Zm00001eb058920_P001'
tid = tid.replace('_T','_P')

if tid in anno.keys():
    print( anno[tid] )
else:
    print( 'no domains')

In [ ]:
# EXAMPLE
# find transcripts that have a specific domain

domain = 'PF08711'

matching_tids = [tid for tid,doms in anno.items() if domain in doms]

print(matching_tids)

In [ ]:
# subroutines to help apply family rules


# return True if the given list of accessions fits the criteria
# all params are lists of strings containing accession names
def matches_family( accessions, required, forbidden ):
    if any( a in forbidden for a in accessions ):
        return False
    r = list(required)
    for a in accessions:
        if a in r:
            r.remove(a)
    if len(r) == 0:
        return True
    return False


# return a list of family names that fit a protein with the given list of accession names
# for each matching family, also return the lists of required and forbidden domains
def get_matching_families( accessions ):
    
    # don't waste any time checking, if no relevant accessions are present
    if not any(a in all_required_domains for a in accessions):
        return []
    
    result = []
    for row in df.index:
        name,required,forbidden = rules_df.loc[row,['GRASSIUS','Required','Forbidden']]
        if matches_family( accessions, required, forbidden ):
            result.append( [name,required,forbidden] )
            
    return result

In [ ]:
# THIS CELL IS SLOW
# apply family rules and build a detailed spreadsheet to explain everything

# you may interrupt this cell and use the partially-completed result

result_df = pd.DataFrame(columns=['transcript_id','accessions','family','required','forbidden'])


print( "applying family rules to transcript annotations...")


# iterate through transcripts
n = len(anno)
for i,(tid,accessions) in enumerate(anno.items()):
    
    #report progress
    if (i%1000) == 0:
        print( f"{i}/{n}" )
        
    # find matching families
    all_matching_families = get_matching_families( accessions )
    
    # for each matching family, add a new row to the final result
    for family_name,required,forbidden in all_matching_families:
        
        new_row_index = len(result_df.index)
        result_df.loc[new_row_index,:] = [str(v) for v in [tid,accessions,family_name,required,forbidden]]

In [ ]:
# save results
result_df.to_csv(output_filepath, index=False)

result_df

In [ ]:
# EXAMPLE
# check results for one family

family = 'MYB'

family_df = result_df[result_df['family'] == family]

family_df

In [ ]:
# EXAMPLE
# check results for one transcript

tid = 'Zm00001eb072200_P002'


transcript_df = result_df[result_df['transcript_id'] == tid]

transcript_df


In [ ]:
# EXAMPLE
# check the domains present in a transcript

tid = 'Zm00001eb325970_P002'
tid = tid.replace('_T','_P')

if tid in anno.keys():
    print( anno[tid] )
else:
    print( 'no domains')